In [1]:
from datetime import datetime, timedelta
import pandas as pd 
# from influxdb_client import InfluxDBClient
from influxdb_client import InfluxDBClient, Point, WritePrecision, WriteOptions 
from influxdb_client.client.write_api import SYNCHRONOUS
import configparser
import pickle

config = configparser.ConfigParser()
config.read('../config.ini')


TOKEN = config['INFLUXDB']['API_KEY']
API_URL = config['INFLUXDB']['API_URL']
ORG = config['INFLUXDB']['ORG']

client = InfluxDBClient(url=API_URL, token=TOKEN, org=ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()


# The defined buckets should be created in influxdb - consult influxdb manuals. 
BUCKET = 'my_bucket'
TEST1 = 'test_bucket1'
TEST2 = 'test_bucket2'

In [2]:
with open('data/20200101-binance_futures-1m.pkl', 'rb') as f:
	data = pickle.load(f)

## Example of Dataframe writing to InfluxDB

In [3]:
sample_data = data[0:100]
sample_data

,open,high,low,close,volume
datetime,,,,,
2020-01-01 00:00:00,7189.43,7190.52,7177.00,7182.44,246.092
2020-01-01 00:01:00,7182.43,7182.44,7178.75,7179.01,70.909
2020-01-01 00:02:00,7179.01,7179.01,7175.25,7177.93,99.420
2020-01-01 00:03:00,7177.77,7182.60,7177.00,7181.11,69.330
2020-01-01 00:04:00,7179.10,7179.10,7172.94,7175.25,97.368
...,...,...,...,...,...
2020-01-01 01:35:00,7213.55,7213.97,7207.74,7207.74,125.766
2020-01-01 01:36:00,7207.76,7212.00,7207.00,7208.74,67.480
2020-01-01 01:37:00,7209.31,7209.47,7207.07,7207.07,43.464


### One column (in this case 'close') should be assigned as value and field, so should not be included in tag_columns

In [4]:
write_api.write(bucket=TEST1, record=sample_data, data_frame_measurement_name='binance',
                data_frame_tag_columns=['open', 'high', 'low', 'volume'])
write_api.close()

In [5]:
query = 'from(bucket: \"{}\") |> range(start:0)'.format(TEST1)
data = client.query_api().query_data_frame(org=ORG, query=query)
data

,result,table,_start,_stop,_time,_value,_field,_measurement,high,low,open,volume
0,_result,0,1970-01-01 00:00:00+00:00,2021-06-23 07:55:27.564137+00:00,2020-01-01 00:59:00+00:00,7171.55,close,binance,7171.61,7171.0,7171.52,28.912
1,_result,1,1970-01-01 00:00:00+00:00,2021-06-23 07:55:27.564137+00:00,2020-01-01 01:00:00+00:00,7171.76,close,binance,7171.77,7171.36,7171.43,9.245
2,_result,2,1970-01-01 00:00:00+00:00,2021-06-23 07:55:27.564137+00:00,2020-01-01 00:58:00+00:00,7171.81,close,binance,7172.92,7171.75,7172.73,3.161
3,_result,3,1970-01-01 00:00:00+00:00,2021-06-23 07:55:27.564137+00:00,2020-01-01 00:57:00+00:00,7173.29,close,binance,7173.33,7171.16,7171.28,26.996
4,_result,4,1970-01-01 00:00:00+00:00,2021-06-23 07:55:27.564137+00:00,2020-01-01 00:20:00+00:00,7171.29,close,binance,7173.5,7171.0,7173.5,63.683
...,...,...,...,...,...,...,...,...,...,...,...,...
95,_result,95,1970-01-01 00:00:00+00:00,2021-06-23 07:55:27.564137+00:00,2020-01-01 01:38:00+00:00,7216.73,close,binance,7216.73,7207.98,7207.98,69.946
96,_result,96,1970-01-01 00:00:00+00:00,2021-06-23 07:55:27.564137+00:00,2020-01-01 01:33:00+00:00,7210.07,close,binance,7217.64,7208.0,7214.83,91.493
97,_result,97,1970-01-01 00:00:00+00:00,2021-06-23 07:55:27.564137+00:00,2020-01-01 01:30:00+00:00,7220.47,close,binance,7220.55,7205.89,7205.89,204.296
98,_result,98,1970-01-01 00:00:00+00:00,2021-06-23 07:55:27.564137+00:00,2020-01-01 01:32:00+00:00,7214.83,close,binance,7222.95,7208.0,7216.9,157.719


## Write data (dataframe) to influxdb

In [9]:
with open('data/20200101-exchange_inflow-block.pkl', 'rb') as f:
	inflow_data = pickle.load(f)

inflow_data.set_index('datetime', inplace=True)
inflow_data

,blockheight,inflow_total,inflow_top10,inflow_mean
datetime,,,,
2020-01-01 00:03:05,610691,339.306123,330.610481,1.368170
2020-01-01 00:07:32,610692,71.728480,71.726044,0.788225
2020-01-01 00:13:34,610693,39.837081,39.779510,0.402395
2020-01-01 00:19:00,610694,69.974007,69.968218,0.874675
2020-01-01 00:37:32,610695,171.701157,170.507771,0.833501
...,...,...,...,...
2021-06-22 07:55:21,688467,613.384863,612.072143,4.172686
2021-06-22 08:25:35,688468,1065.423847,1003.685588,3.170904
2021-06-22 08:30:43,688469,354.832382,302.683069,1.271801


In [11]:
write_api.write(bucket=TEST2, record=inflow_data, data_frame_measurement_name='exchange_inflow',
                data_frame_tag_columns=['inflow_top10', 'inflow_mean', 'blockheight'])
write_api.close()

In [12]:
query = 'from(bucket: \"{}\") |> range(start:0)'.format(TEST2)
data = client.query_api().query_data_frame(org=ORG, query=query)
data

,result,table,_start,_stop,_time,_value,_field,_measurement,blockheight,inflow_mean,inflow_top10
0,_result,0,1970-01-01 00:00:00+00:00,2021-06-23 07:58:27.730436+00:00,2020-01-01 00:03:05+00:00,339.306123,inflow_total,exchange_inflow,610691,1.36816985,330.6104809
1,_result,1,1970-01-01 00:00:00+00:00,2021-06-23 07:58:27.730436+00:00,2020-01-01 00:07:32+00:00,71.728480,inflow_total,exchange_inflow,610692,0.78822506,71.72604394
2,_result,2,1970-01-01 00:00:00+00:00,2021-06-23 07:58:27.730436+00:00,2020-01-01 00:13:34+00:00,39.837081,inflow_total,exchange_inflow,610693,0.40239476,39.77950962
3,_result,3,1970-01-01 00:00:00+00:00,2021-06-23 07:58:27.730436+00:00,2020-01-01 00:19:00+00:00,69.974007,inflow_total,exchange_inflow,610694,0.87467508,69.96821796
4,_result,4,1970-01-01 00:00:00+00:00,2021-06-23 07:58:27.730436+00:00,2020-01-01 00:37:32+00:00,171.701157,inflow_total,exchange_inflow,610695,0.83350076,170.50777056
...,...,...,...,...,...,...,...,...,...,...,...
77776,_result,77776,1970-01-01 00:00:00+00:00,2021-06-23 07:58:27.730436+00:00,2021-06-22 07:55:21+00:00,613.384863,inflow_total,exchange_inflow,688467,4.17268614,612.07214278
77777,_result,77777,1970-01-01 00:00:00+00:00,2021-06-23 07:58:27.730436+00:00,2021-06-22 08:25:35+00:00,1065.423847,inflow_total,exchange_inflow,688468,3.17090431,1003.68558808
77778,_result,77778,1970-01-01 00:00:00+00:00,2021-06-23 07:58:27.730436+00:00,2021-06-22 08:30:43+00:00,354.832382,inflow_total,exchange_inflow,688469,1.27180065,302.68306899
77779,_result,77779,1970-01-01 00:00:00+00:00,2021-06-23 07:58:27.730436+00:00,2021-06-22 08:38:41+00:00,633.441961,inflow_total,exchange_inflow,688470,3.53878191,547.75560442
